In [1]:
import os
import json
from tqdm import tqdm
import pandas as pd
import math
import numpy as np
from collections import defaultdict

import pickle
from collections import Counter

In [2]:
## nodes: opinion, opinion_cluster, court, docket
## opinion features: plain_text
## opinion_cluster features: syllabus, judges, case_name, attorneys
## court: full_name, start_date, end_date, citation_string
## docket: pacer_case_id, case_name_full

## (parenthetical): text

In [4]:
raw_data_dir="/home/ubuntu/quic-efs/user/bowenjin/llm-graph-plugin/data/raw_data/legal"
save_dir="/home/ubuntu/quic-efs/user/bowenjin/llm-graph-plugin/data/processed_data/legal"

In [5]:
## read raw data files

opinion_cluster_raw_data = pd.read_csv(os.path.join(raw_data_dir, 'opinion-clusters-2023-08-31.csv'))
court_raw_data = pd.read_csv(os.path.join(raw_data_dir, 'courts-2023-08-31.csv'))
citation_raw_data = pd.read_csv(os.path.join(raw_data_dir, 'citation-map-2023-08-31.csv'))
parentheticals_raw_data = pd.read_csv(os.path.join(raw_data_dir, 'parentheticals-2023-08-31.csv'))
dockets_raw_data = pd.read_csv(os.path.join(raw_data_dir, 'dockets-2023-08-31.csv'))
opinion_raw_data = pd.read_csv(os.path.join(raw_data_dir, 'opinions-2023-08-31.csv'))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


/tmp/ipykernel_2717714/1653722691.py:3: DtypeWarning: Columns (10,17,18,19,20,21,22,23,24,25,26,29,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  opinion_cluster_raw_data = pd.read_csv(os.path.join(raw_data_dir, 'opinion-clusters-2023-08-31.csv'))
/tmp/ipykernel_2717714/1653722691.py:7: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,19,21,23,24,25,26,27,28,29,30,31,32,33,35,36,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  dockets_raw_data = pd.read_csv(os.path.join(raw_data_dir, 'dockets-2023-08-31.csv'))
/tmp/ipykernel_2717714/1653722691.py:8: DtypeWarning: Columns (7,9,10,11,12,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  opinion_raw_data = pd.read_csv(os.path.join(raw_data_dir, 'opinions-2023-08-31.csv'))


In [6]:
print(len(opinion_cluster_raw_data), len(dockets_raw_data), len(citation_raw_data), len(parentheticals_raw_data), len(opinion_raw_data))

8961877 66164609 29305192 1206771 9592141


In [7]:
opinion_raw_data.head(n=5)

,id,date_created,date_modified,author_str,per_curiam,joined_by_str,type,sha1,page_count,download_url,local_path,plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,extracted_by_ocr,author_id,cluster_id
0,7344185,2022-07-26 00:22:04.338683+00,2022-07-26 00:22:04.338698+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<author id=""b1161-1...",NaN,t,NaN,7424457
1,7344186,2022-07-26 00:22:07.267308+00,2022-07-26 00:22:07.267322+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""AjmI"">No opi...",NaN,t,NaN,7424458
2,7344187,2022-07-26 00:22:08.299694+00,2022-07-26 00:22:08.299713+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""ATKK"">No opi...",NaN,t,NaN,7424459
3,7344188,2022-07-26 00:22:09.289318+00,2022-07-26 00:22:09.289335+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""AeGF"">No opi...",NaN,t,NaN,7424460
4,7344189,2022-07-26 00:22:10.242719+00,2022-07-26 00:22:10.242737+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""AnO"">No opin...",NaN,t,NaN,7424461


In [9]:
opinion_cluster_raw_data.head(n=5)

,id,date_created,date_modified,judges,date_filed,date_filed_is_approximate,slug,case_name_short,case_name,case_name_full,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,source,procedural_history,attorneys,nature_of_suit,posture,syllabus,headnotes,summary,disposition,history,other_dates,cross_reference,correction,citation_count,precedential_status,date_blocked,blocked,filepath_json_harvard,docket_id,arguments,headmatter
0,8599951,2022-11-23 15:17:50.049155+00,2022-11-23 15:17:50.049164+00,Eichaedson,1875-12-15,t,kaufman-v-united-states,Kaufman,Kaufman v. United States,Sebastian Kaufman v. United States,NaN,NaN,NaN,NaN,U,NaN,"Mr. P. P. Pye for the claimant., Mr. Assistant...",NaN,NaN,"<p>In May, 1873, a brewer pays to the collecto...",NaN,<p>\nOn the Proofs.\n</p> <p>The Reporters’ st...,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,f,/storage/harvard_corpus/law.free.cap.ct-cl.11/...,65862636,NaN,NaN
1,8599952,2022-11-23 15:17:50.371002+00,2022-11-23 15:17:50.371011+00,Nott,1875-12-15,t,norton-v-united-states,Norton,Norton v. United States,"E. E. Norton, assignee of De Bow & Co. v. Unit...",NaN,NaN,NaN,NaN,U,NaN,"Messrs. E. <& A. O. Jarán for' the claimant:, ...",NaN,NaN,"<p>On the 2ith June, 1865, restrictions on com...",NaN,<p>\nOn the Proofs.\n</p> <p>The Reporters’ st...,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,f,/storage/harvard_corpus/law.free.cap.ct-cl.11/...,65862637,NaN,NaN
2,8599953,2022-11-23 15:17:50.65916+00,2022-11-23 15:17:50.65917+00,"Losing, Nott",1875-12-15,t,wood-v-united-states,Wood,Wood v. United States,Albert Wood v. United States,NaN,NaN,NaN,NaN,U,NaN,"Mr. Enoch Totten for the claimant:, Mr. John S...",NaN,NaN,<p>The claimant agrees to furnish at Fort Brow...,NaN,<p>\nOn the Proofs.\n</p> <p>The Reporters' st...,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,f,/storage/harvard_corpus/law.free.cap.ct-cl.11/...,65862638,NaN,NaN
3,8599954,2022-11-23 15:17:50.915725+00,2022-11-23 15:17:50.915739+00,Nott,1875-12-15,t,chilson-v-united-states,Chilson,Chilson v. United States,George W. Chilson v. United States,NaN,NaN,NaN,NaN,U,NaN,"Mr. JI. H. Paine for the claimant., Mr. A. 1)....",NaN,NaN,"<p>An officer, at Ms own request, is ordered h...",NaN,<p>\nOn the Proofs.\n</p> <p>The Reporters’ st...,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,f,/storage/harvard_corpus/law.free.cap.ct-cl.11/...,65862639,NaN,NaN
4,8599985,2022-11-23 15:17:59.363485+00,2022-11-23 15:17:59.363496+00,NaN,1875-12-15,t,cole-v-united-states,Cole,Cole v. United States,Francis Cole v. United States,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,f,/storage/harvard_corpus/law.free.cap.ct-cl.11/...,65862670,NaN,NaN


In [10]:
court_raw_data.head(n=5)

,id,pacer_court_id,pacer_has_rss_feed,pacer_rss_entry_types,fjc_court_id,date_modified,in_use,has_opinion_scraper,has_oral_argument_scraper,position,citation_string,short_name,full_name,url,start_date,end_date,jurisdiction,notes
0,nc,NaN,NaN,NaN,NaN,2016-09-08 20:38:41.131652+00,t,t,f,366.97,N.C.,Supreme Court of North Carolina,Supreme Court of North Carolina,http://www.nccourts.org/courts/appellate/supreme/,1799-01-01,NaN,S,Created by Lawbox\r\nStart date: http://www.ca...
1,ag,NaN,NaN,NaN,NaN,2017-01-27 22:35:06.13121+00,t,t,f,113.04,Op. Att’y Gen.,Attorneys General,United States Attorneys General,https://www.justice.gov/olc/opinions-main,NaN,NaN,FS,NaN
2,minnag,NaN,NaN,NaN,NaN,2017-02-01 17:56:35.255825+00,t,t,f,365.40,Minn. Att'y Gen.,Minnesota Attorney General Reports,Minnesota Attorney General Reports,http://www.ag.state.mn.us/office/opinions/DATE...,NaN,NaN,SAG,NaN
3,arizsuper,NaN,NaN,NaN,NaN,2017-10-02 21:06:41.874279+00,f,f,f,322.30,Ariz. Super. Ct.,Arizona Superior Court,Arizona Superior Court,http://www.azcourts.gov/AZ-Courts/Superior-Court,NaN,NaN,ST,NaN
4,vtsuperct,NaN,NaN,NaN,NaN,2018-04-23 23:31:46.174336+00,t,t,f,369.25,Vt. Super. Ct.,Vermont Superior Court,Vermont Superior Court,https://www.vermontjudiciary.org/court-divisions,NaN,NaN,SA,NaN


In [11]:
dockets_raw_data.head(n=5)

,id,date_created,date_modified,source,appeal_from_str,assigned_to_str,referred_to_str,panel_str,date_last_index,date_cert_granted,date_cert_denied,date_argued,date_reargued,date_reargument_denied,date_filed,date_terminated,date_last_filing,case_name_short,case_name,case_name_full,slug,docket_number,docket_number_core,pacer_case_id,cause,nature_of_suit,jury_demand,jurisdiction_type,appellate_fee_status,appellate_case_type_information,mdl_status,filepath_local,filepath_ia,filepath_ia_json,ia_upload_failure_count,ia_needs_upload,ia_date_first_change,view_count,date_blocked,blocked,appeal_from_id,assigned_to_id,court_id,idb_data_id,originating_court_information_id,referred_to_id
0,67541426,2023-06-29 21:34:57.392765+00,2023-08-30 12:03:17.814757+00,1,NaN,NaN,NaN,NaN,2023-07-02 12:23:33.338483+00,NaN,NaN,NaN,NaN,NaN,2023-06-29,NaN,2023-08-29,Sherman N. Love,Sherman N. Love,NaN,sherman-n-love,3-23-11110,NaN,194586.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,2023-06-29 21:34:57.391819+00,0,2023-06-29,t,NaN,9006.0,wiwb,NaN,NaN,NaN
1,67511287,2023-06-19 23:00:55.271191+00,2023-08-04 01:36:18.764079+00,1,NaN,NaN,NaN,NaN,2023-06-23 13:07:13.14988+00,NaN,NaN,NaN,NaN,NaN,2023-06-19,NaN,2023-08-02,Colleen R. Rogers,Colleen R. Rogers,NaN,colleen-r-rogers,1-23-11015,NaN,194489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,2023-06-19 23:00:55.269531+00,0,2023-06-19,t,NaN,NaN,wiwb,NaN,NaN,NaN
2,67411372,2023-05-21 00:32:20.623644+00,2023-08-31 09:34:02.099519+00,1,NaN,NaN,NaN,NaN,2023-05-23 12:55:10.014442+00,NaN,NaN,NaN,NaN,NaN,2023-05-20,NaN,2023-08-30,NaN,Christian James Mattice Leier,NaN,christian-james-mattice-leier,1-23-10835,NaN,194303.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,2023-05-21 00:32:20.622066+00,0,2023-05-20,t,NaN,NaN,wiwb,NaN,NaN,NaN
3,65748136,2022-11-10 23:33:18.999178+00,2023-08-30 23:03:53.509555+00,1,NaN,NaN,NaN,NaN,2022-11-11 14:53:35.678958+00,NaN,NaN,NaN,NaN,NaN,2022-11-10,NaN,2023-08-29,Cesar Iturra,Cesar Iturra,NaN,cesar-iturra,22-40464,22040464.0,169001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,2022-11-10 23:33:18.996846+00,0,2022-11-10,t,NaN,8781.0,idb,NaN,NaN,NaN
4,7703226,2018-08-21 14:48:56.208271+00,2023-08-24 16:06:00.281194+00,1,NaN,NaN,NaN,NaN,2022-08-13 19:26:26.052504+00,NaN,NaN,NaN,NaN,NaN,2018-08-21,NaN,2023-08-23,Anthony Freeman,Anthony Freeman,NaN,anthony-freeman,3:18-bk-32539,1832539.0,451842.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://archive.org/download/gov.uscourts.tneb...,NaN,t,2018-11-05 13:03:13.025446+00,0,2018-08-21,t,NaN,8953.0,tneb,NaN,NaN,NaN


In [12]:
parentheticals_raw_data.head(n=5)

,id,text,score,described_opinion_id,describing_opinion_id,group_id
0,2618061,“The imposition of lifetime parole is not tant...,0.464000,1273120,4643569,2287586
1,1598538,"""the [trial] court must make factual findings ...",0.526035,791082,2119488,2098010
2,1946880,explaining that the phrase change of circumsta...,0.800000,3708514,2697255,2610651
3,1335394,separate convictions for simple possession and...,0.509328,1163906,1230126,2098026
4,239069,applying traditional debt/equity factors to re...,0.687856,1689935,22187,2193521


In [13]:
citation_raw_data.head(n=5)

,id,depth,cited_opinion_id,citing_opinion_id
0,189242932,1,438723,4539950
1,189242933,2,1732,4539950
2,189242934,1,180584,4539950
3,189242935,1,799785,4539950
4,189242936,1,108427,4539950


In [14]:
opinion_raw_data.head(n=5)

,id,date_created,date_modified,author_str,per_curiam,joined_by_str,type,sha1,page_count,download_url,local_path,plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,extracted_by_ocr,author_id,cluster_id
0,7344185,2022-07-26 00:22:04.338683+00,2022-07-26 00:22:04.338698+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<author id=""b1161-1...",NaN,t,NaN,7424457
1,7344186,2022-07-26 00:22:07.267308+00,2022-07-26 00:22:07.267322+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""AjmI"">No opi...",NaN,t,NaN,7424458
2,7344187,2022-07-26 00:22:08.299694+00,2022-07-26 00:22:08.299713+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""ATKK"">No opi...",NaN,t,NaN,7424459
3,7344188,2022-07-26 00:22:09.289318+00,2022-07-26 00:22:09.289335+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""AeGF"">No opi...",NaN,t,NaN,7424460
4,7344189,2022-07-26 00:22:10.242719+00,2022-07-26 00:22:10.242737+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""AnO"">No opin...",NaN,t,NaN,7424461


In [15]:
opinion_raw_data.loc[opinion_raw_data['id'] == 4539950]['html'].values[0]

nan

In [142]:
# plain_text, html, html_lawbox, html_columbia, html_anon_2020, xml_harvard, html_with_citations
all_set = set()
for idd, row in tqdm(opinion_raw_data.iterrows()):
    values = []
    values.append(isinstance(row['plain_text'], str))
    values.append(isinstance(row['html'], str))
    values.append(isinstance(row['html_lawbox'], str))
    values.append(isinstance(row['html_columbia'], str))
    values.append(isinstance(row['html_anon_2020'], str))
    values.append(isinstance(row['xml_harvard'], str))
    values.append(isinstance(row['html_with_citations'], str))
    all_set.add(tuple(values))
print(all_set)

9592141it [08:16, 19325.35it/s]

{(False, False, False, True, False, False, False), (True, False, True, False, False, False, False), (False, False, False, False, True, False, True), (False, True, False, False, False, True, True), (False, False, True, False, False, False, False), (False, True, True, False, False, False, True), (True, False, False, False, False, True, False), (True, True, False, False, False, False, False), (True, False, True, False, False, True, True), (True, False, False, False, False, False, True), (True, True, True, False, False, True, True), (True, False, False, False, True, False, True), (False, False, False, False, False, False, True), (False, False, False, False, False, True, False), (False, False, True, False, False, True, True), (False, False, False, False, True, False, False), (False, True, False, False, False, True, False), (False, True, False, False, False, False, True), (False, True, True, False, False, False, False), (True, True, False, False, False, True, True), (True, False, False, Fals

In [156]:
for idd, row in tqdm(opinion_raw_data.iterrows()):
    values = []
    values.append(isinstance(row['plain_text'], str))
    values.append(isinstance(row['html'], str))
    values.append(isinstance(row['html_lawbox'], str))
    values.append(isinstance(row['html_columbia'], str))
    values.append(isinstance(row['html_anon_2020'], str))
    values.append(isinstance(row['xml_harvard'], str))
    values.append(isinstance(row['html_with_citations'], str))
    if isinstance(row['html_columbia'], str) and isinstance(row['html_with_citations'], str):
        print(values)
        print(row['html_columbia'])
        raise ValueError('stop')

2692094it [02:29, 18056.83it/s]

[False, False, False, True, False, False, True]
<p>Plaintiffs exchanged with defendants some farm land, situate near Scottsdale, Maricopa county, Arizona, for the Woodford hotel, situate in Phoenix, Arizona. This suit is brought for damages for deceit the defendants are alleged to have practiced on plaintiffs in such trade. The deceit charged is that defendants represented before and at the time of the exchange of deeds that the land (consisting of 176 acres) "was entitled, as of right, and appurtenant to the ownership thereof, to permanent water"; that plaintiffs relied upon such representation, and believed it to be true, and so believing gave defendants a deed to the hotel property; that the representation was false and untrue, and so known to defendants at the time, the truth being the land had only temporary "war order" water.</p>
<p>The defendants admitted in their answer that they represented "that 40 acres of said land was entitled to a permanent water right, and that the rest 

ValueError: stop

In [144]:
for t in all_set:
    if not t[0]:
        print(t)

(False, False, False, True, False, False, False)
(False, False, False, False, True, False, True)
(False, True, False, False, False, True, True)
(False, False, True, False, False, False, False)
(False, True, True, False, False, False, True)
(False, False, False, False, False, False, True)
(False, False, False, False, False, True, False)
(False, False, True, False, False, True, True)
(False, False, False, False, True, False, False)
(False, True, False, False, False, True, False)
(False, True, False, False, False, False, True)
(False, True, True, False, False, False, False)
(False, False, True, False, False, True, False)
(False, False, False, False, False, False, False)
(False, False, False, True, False, False, True)
(False, True, False, False, False, False, False)
(False, True, True, False, False, True, True)
(False, False, True, False, False, False, True)


In [134]:
math.isnan(row['plain_text'])

True

In [164]:
opinion_cluster_raw_data.head(n=5)

,id,date_created,date_modified,judges,date_filed,date_filed_is_approximate,slug,case_name_short,case_name,case_name_full,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,source,procedural_history,attorneys,nature_of_suit,posture,syllabus,headnotes,summary,disposition,history,other_dates,cross_reference,correction,citation_count,precedential_status,date_blocked,blocked,filepath_json_harvard,docket_id,arguments,headmatter
0,8599951,2022-11-23 15:17:50.049155+00,2022-11-23 15:17:50.049164+00,Eichaedson,1875-12-15,t,kaufman-v-united-states,Kaufman,Kaufman v. United States,Sebastian Kaufman v. United States,NaN,NaN,NaN,NaN,U,NaN,"Mr. P. P. Pye for the claimant., Mr. Assistant...",NaN,NaN,"<p>In May, 1873, a brewer pays to the collecto...",NaN,<p>\nOn the Proofs.\n</p> <p>The Reporters’ st...,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,f,/storage/harvard_corpus/law.free.cap.ct-cl.11/...,65862636,NaN,NaN
1,8599952,2022-11-23 15:17:50.371002+00,2022-11-23 15:17:50.371011+00,Nott,1875-12-15,t,norton-v-united-states,Norton,Norton v. United States,"E. E. Norton, assignee of De Bow & Co. v. Unit...",NaN,NaN,NaN,NaN,U,NaN,"Messrs. E. <& A. O. Jarán for' the claimant:, ...",NaN,NaN,"<p>On the 2ith June, 1865, restrictions on com...",NaN,<p>\nOn the Proofs.\n</p> <p>The Reporters’ st...,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,f,/storage/harvard_corpus/law.free.cap.ct-cl.11/...,65862637,NaN,NaN
2,8599953,2022-11-23 15:17:50.65916+00,2022-11-23 15:17:50.65917+00,"Losing, Nott",1875-12-15,t,wood-v-united-states,Wood,Wood v. United States,Albert Wood v. United States,NaN,NaN,NaN,NaN,U,NaN,"Mr. Enoch Totten for the claimant:, Mr. John S...",NaN,NaN,<p>The claimant agrees to furnish at Fort Brow...,NaN,<p>\nOn the Proofs.\n</p> <p>The Reporters' st...,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,f,/storage/harvard_corpus/law.free.cap.ct-cl.11/...,65862638,NaN,NaN
3,8599954,2022-11-23 15:17:50.915725+00,2022-11-23 15:17:50.915739+00,Nott,1875-12-15,t,chilson-v-united-states,Chilson,Chilson v. United States,George W. Chilson v. United States,NaN,NaN,NaN,NaN,U,NaN,"Mr. JI. H. Paine for the claimant., Mr. A. 1)....",NaN,NaN,"<p>An officer, at Ms own request, is ordered h...",NaN,<p>\nOn the Proofs.\n</p> <p>The Reporters’ st...,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,f,/storage/harvard_corpus/law.free.cap.ct-cl.11/...,65862639,NaN,NaN
4,8599985,2022-11-23 15:17:59.363485+00,2022-11-23 15:17:59.363496+00,NaN,1875-12-15,t,cole-v-united-states,Cole,Cole v. United States,Francis Cole v. United States,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,f,/storage/harvard_corpus/law.free.cap.ct-cl.11/...,65862670,NaN,NaN


In [165]:
dockets_raw_data.head(n=5)

,id,date_created,date_modified,source,appeal_from_str,assigned_to_str,referred_to_str,panel_str,date_last_index,date_cert_granted,date_cert_denied,date_argued,date_reargued,date_reargument_denied,date_filed,date_terminated,date_last_filing,case_name_short,case_name,case_name_full,slug,docket_number,docket_number_core,pacer_case_id,cause,nature_of_suit,jury_demand,jurisdiction_type,appellate_fee_status,appellate_case_type_information,mdl_status,filepath_local,filepath_ia,filepath_ia_json,ia_upload_failure_count,ia_needs_upload,ia_date_first_change,view_count,date_blocked,blocked,appeal_from_id,assigned_to_id,court_id,idb_data_id,originating_court_information_id,referred_to_id
0,67541426,2023-06-29 21:34:57.392765+00,2023-08-30 12:03:17.814757+00,1,NaN,NaN,NaN,NaN,2023-07-02 12:23:33.338483+00,NaN,NaN,NaN,NaN,NaN,2023-06-29,NaN,2023-08-29,Sherman N. Love,Sherman N. Love,NaN,sherman-n-love,3-23-11110,NaN,194586.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,2023-06-29 21:34:57.391819+00,0,2023-06-29,t,NaN,9006.0,wiwb,NaN,NaN,NaN
1,67511287,2023-06-19 23:00:55.271191+00,2023-08-04 01:36:18.764079+00,1,NaN,NaN,NaN,NaN,2023-06-23 13:07:13.14988+00,NaN,NaN,NaN,NaN,NaN,2023-06-19,NaN,2023-08-02,Colleen R. Rogers,Colleen R. Rogers,NaN,colleen-r-rogers,1-23-11015,NaN,194489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,2023-06-19 23:00:55.269531+00,0,2023-06-19,t,NaN,NaN,wiwb,NaN,NaN,NaN
2,67411372,2023-05-21 00:32:20.623644+00,2023-08-31 09:34:02.099519+00,1,NaN,NaN,NaN,NaN,2023-05-23 12:55:10.014442+00,NaN,NaN,NaN,NaN,NaN,2023-05-20,NaN,2023-08-30,NaN,Christian James Mattice Leier,NaN,christian-james-mattice-leier,1-23-10835,NaN,194303.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,2023-05-21 00:32:20.622066+00,0,2023-05-20,t,NaN,NaN,wiwb,NaN,NaN,NaN
3,65748136,2022-11-10 23:33:18.999178+00,2023-08-30 23:03:53.509555+00,1,NaN,NaN,NaN,NaN,2022-11-11 14:53:35.678958+00,NaN,NaN,NaN,NaN,NaN,2022-11-10,NaN,2023-08-29,Cesar Iturra,Cesar Iturra,NaN,cesar-iturra,22-40464,22040464.0,169001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,2022-11-10 23:33:18.996846+00,0,2022-11-10,t,NaN,8781.0,idb,NaN,NaN,NaN
4,7703226,2018-08-21 14:48:56.208271+00,2023-08-24 16:06:00.281194+00,1,NaN,NaN,NaN,NaN,2022-08-13 19:26:26.052504+00,NaN,NaN,NaN,NaN,NaN,2018-08-21,NaN,2023-08-23,Anthony Freeman,Anthony Freeman,NaN,anthony-freeman,3:18-bk-32539,1832539.0,451842.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://archive.org/download/gov.uscourts.tneb...,NaN,t,2018-11-05 13:03:13.025446+00,0,2018-08-21,t,NaN,8953.0,tneb,NaN,NaN,NaN


In [166]:
court_raw_data.head(n=5)

,id,pacer_court_id,pacer_has_rss_feed,pacer_rss_entry_types,fjc_court_id,date_modified,in_use,has_opinion_scraper,has_oral_argument_scraper,position,citation_string,short_name,full_name,url,start_date,end_date,jurisdiction,notes
0,nc,NaN,NaN,NaN,NaN,2016-09-08 20:38:41.131652+00,t,t,f,366.97,N.C.,Supreme Court of North Carolina,Supreme Court of North Carolina,http://www.nccourts.org/courts/appellate/supreme/,1799-01-01,NaN,S,Created by Lawbox\r\nStart date: http://www.ca...
1,ag,NaN,NaN,NaN,NaN,2017-01-27 22:35:06.13121+00,t,t,f,113.04,Op. Att’y Gen.,Attorneys General,United States Attorneys General,https://www.justice.gov/olc/opinions-main,NaN,NaN,FS,NaN
2,minnag,NaN,NaN,NaN,NaN,2017-02-01 17:56:35.255825+00,t,t,f,365.40,Minn. Att'y Gen.,Minnesota Attorney General Reports,Minnesota Attorney General Reports,http://www.ag.state.mn.us/office/opinions/DATE...,NaN,NaN,SAG,NaN
3,arizsuper,NaN,NaN,NaN,NaN,2017-10-02 21:06:41.874279+00,f,f,f,322.30,Ariz. Super. Ct.,Arizona Superior Court,Arizona Superior Court,http://www.azcourts.gov/AZ-Courts/Superior-Court,NaN,NaN,ST,NaN
4,vtsuperct,NaN,NaN,NaN,NaN,2018-04-23 23:31:46.174336+00,t,t,f,369.25,Vt. Super. Ct.,Vermont Superior Court,Vermont Superior Court,https://www.vermontjudiciary.org/court-divisions,NaN,NaN,SA,NaN


In [185]:
opinion_raw_data.head(n=5)

,id,date_created,date_modified,author_str,per_curiam,joined_by_str,type,sha1,page_count,download_url,local_path,plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,extracted_by_ocr,author_id,cluster_id
0,7344185,2022-07-26 00:22:04.338683+00,2022-07-26 00:22:04.338698+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<author id=""b1161-1...",NaN,t,NaN,7424457
1,7344186,2022-07-26 00:22:07.267308+00,2022-07-26 00:22:07.267322+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""AjmI"">No opi...",NaN,t,NaN,7424458
2,7344187,2022-07-26 00:22:08.299694+00,2022-07-26 00:22:08.299713+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""ATKK"">No opi...",NaN,t,NaN,7424459
3,7344188,2022-07-26 00:22:09.289318+00,2022-07-26 00:22:09.289335+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""AeGF"">No opi...",NaN,t,NaN,7424460
4,7344189,2022-07-26 00:22:10.242719+00,2022-07-26 00:22:10.242737+00,NaN,f,NaN,020lead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<opinion type=""majority"">\n<p id=""AnO"">No opin...",NaN,t,NaN,7424461


In [171]:
opinion_raw_data.loc[opinion_raw_data['id'] == 438723]

,id,date_created,date_modified,author_str,per_curiam,joined_by_str,type,sha1,page_count,download_url,local_path,plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,extracted_by_ocr,author_id,cluster_id
5244235,438723,2011-08-23 09:35:20+00,2020-02-22 06:00:20.095567+00,NaN,f,NaN,010combined,5b53797c0298ef567184cbe6b49cda72fbed44af,NaN,http://bulk.resource.org/courts.gov/c/F2/738/7...,NaN,NaN,"<p class=""case_cite"">738 F.2d 863</p>\n <p ...",NaN,NaN,NaN,NaN,"<p class=""case_cite""><span class=""citation no-...",f,NaN,438723


In [168]:
citation_raw_data.head(n=5)

,id,depth,cited_opinion_id,citing_opinion_id
0,189242932,1,438723,4539950
1,189242933,2,1732,4539950
2,189242934,1,180584,4539950
3,189242935,1,799785,4539950
4,189242936,1,108427,4539950


In [173]:
max(citation_raw_data['depth'].tolist())

617

In [8]:
## construct book node dictionary
## opinion features: date_created, date_modified, text (plain_text, html, html_lawbox, html_columbia, html_anon_2020, xml_harvard, html_with_citations)
## opinion edges: citing opinions, cited opinions, opinion cluster

## opinion_cluster features: date_created, date_modified, judges, date_filed, slug, case_name_short, case_name, case_name_full, scdb_id, scdb_decision_direction, scdb_votes_majority, scdb_votes_minority, attorneys, syllabus, headnotes, summary
## opinion_cluster edges: opinions, docket

## docket features: date_created, date_modified, date_last_index, date_filed, date_last_filing, case_name_short	case_name, case_name_full, slug, docket_number, docket_number_core, pacer_case_id, ia_date_first_change, date_blocked
## docket edges: opinion cluster, court

## court: date_modified, position, citation_string, short_name, full_name, url, start_date, end_date, jurisdiction, notes
## court edges: docket


opinion_nodes = {}
opinion_cluster_nodes = {}
docket_nodes = {}
court_nodes = {}

opinion_cluster2opinions = defaultdict(list)
docket2opinion_clusters = defaultdict(list)
court2docket = defaultdict(list)

# opinions nodes
for idd, opinion_row in tqdm(opinion_raw_data.iterrows()):

    opinion_nodes[str(opinion_row['id'])] = {'features': {}, 'neighbors': {}}
    ## add features
    #opinion_nodes[str(opinion_row['id'])]['features']['date_created'] = opinion_row['date_created']
    #opinion_nodes[str(opinion_row['id'])]['features']['date_modified'] = opinion_row['date_modified']
    #opinion_nodes[str(opinion_row['id'])]['features']['xml_harvard'] = opinion_row['xml_harvard'] if isinstance(opinion_row['xml_harvard'], str) else ''
    
    if isinstance(opinion_row['plain_text'], str):
        opinion_nodes[str(opinion_row['id'])]['features']['plain_text'] = opinion_row['plain_text']
    elif isinstance(opinion_row['html'], str):
        opinion_nodes[str(opinion_row['id'])]['features']['plain_text'] = opinion_row['html']
    elif isinstance(opinion_row['html_lawbox'], str):
        opinion_nodes[str(opinion_row['id'])]['features']['plain_text'] = opinion_row['html_lawbox']
    elif isinstance(opinion_row['html_columbia'], str):
        opinion_nodes[str(opinion_row['id'])]['features']['plain_text'] = opinion_row['html_columbia']
    elif isinstance(opinion_row['html_anon_2020'], str):
        opinion_nodes[str(opinion_row['id'])]['features']['plain_text'] = opinion_row['html_anon_2020']
    elif isinstance(opinion_row['html_with_citations'], str):
        opinion_nodes[str(opinion_row['id'])]['features']['plain_text'] = opinion_row['html_with_citations']
    else:
        opinion_nodes[str(opinion_row['id'])]['features']['plain_text'] = ''

    ## add neighbors
    opinion_nodes[str(opinion_row['id'])]['neighbors']['opinion_cluster'] = [str(opinion_row['cluster_id'])]
    opinion_nodes[str(opinion_row['id'])]['neighbors']['reference'] = defaultdict(list) # key is number of citation, value is opinion id
    opinion_nodes[str(opinion_row['id'])]['neighbors']['cited_by'] = defaultdict(list)
    opinion_cluster2opinions[str(opinion_row['cluster_id'])].append(str(opinion_row['id']))

miss_opinion_in_citation = 0
## add neighbors
for idd, citation_row in tqdm(citation_raw_data.iterrows()):
    ## add cited_by
    if str(citation_row['cited_opinion_id']) not in opinion_nodes or str(citation_row['citing_opinion_id']) not in opinion_nodes:
        miss_opinion_in_citation += 1
        continue
    opinion_nodes[str(citation_row['cited_opinion_id'])]['neighbors']['cited_by'][str(citation_row['depth'])].append(str(citation_row['citing_opinion_id']))
    opinion_nodes[str(citation_row['citing_opinion_id'])]['neighbors']['reference'][str(citation_row['depth'])].append(str(citation_row['cited_opinion_id']))
print(f'Miss citation cnt:{miss_opinion_in_citation}')

# opinion cluster nodes
for idd, opinion_cluster_row in tqdm(opinion_cluster_raw_data.iterrows()):
    opinion_cluster_nodes[str(opinion_cluster_row['id'])] = {'features': {}, 'neighbors': {}}
    ## add features
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['date_created'] = opinion_cluster_row['date_created']
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['date_modified'] = opinion_cluster_row['date_modified']
    opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['judges'] = opinion_cluster_row['judges']
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['date_filed'] = opinion_cluster_row['date_filed']
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['slug'] = opinion_cluster_row['slug']
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['case_name_short'] = opinion_cluster_row['case_name_short']
    opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['case_name'] = opinion_cluster_row['case_name']
    opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['case_name_full'] = opinion_cluster_row['case_name_full']
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['scdb_id'] = opinion_cluster_row['scdb_id']
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['scdb_decision_direction'] = opinion_cluster_row['scdb_decision_direction']
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['scdb_votes_majority'] = opinion_cluster_row['scdb_votes_majority']
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['scdb_votes_minority'] = opinion_cluster_row['scdb_votes_minority']
    opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['attorneys'] = opinion_cluster_row['attorneys']
    opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['syllabus'] = opinion_cluster_row['syllabus']
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['headnotes'] = opinion_cluster_row['headnotes']
    #opinion_cluster_nodes[str(opinion_cluster_row['id'])]['features']['summary'] = opinion_cluster_row['summary']
    ## add neighbors
    opinion_cluster_nodes[str(opinion_cluster_row['id'])]['neighbors']['opinion'] = opinion_cluster2opinions[str(opinion_cluster_row['id'])] if str(opinion_cluster_row['id']) in opinion_cluster2opinions else []
    opinion_cluster_nodes[str(opinion_cluster_row['id'])]['neighbors']['docket'] = [str(opinion_cluster_row['docket_id'])]
    docket2opinion_clusters[str(opinion_cluster_row['docket_id'])].append(str(opinion_cluster_row['id']))

# add docket nodes
for idd, docket_row in tqdm(dockets_raw_data.iterrows()):
    docket_nodes[str(docket_row['id'])] = {'features': {}, 'neighbors': {}}
    ## add features
    #docket_nodes[str(docket_row['id'])]['features']['date_created'] = docket_row['date_created']
    #docket_nodes[str(docket_row['id'])]['features']['date_modified'] = docket_row['date_modified']
    #docket_nodes[str(docket_row['id'])]['features']['date_last_index'] = docket_row['date_last_index']
    #docket_nodes[str(docket_row['id'])]['features']['date_filed'] = docket_row['date_filed']
    #docket_nodes[str(docket_row['id'])]['features']['date_last_filing'] = docket_row['date_last_filing']
    #docket_nodes[str(docket_row['id'])]['features']['case_name_short'] = docket_row['case_name_short']
    #docket_nodes[str(docket_row['id'])]['features']['case_name'] = docket_row['case_name']
    docket_nodes[str(docket_row['id'])]['features']['case_name_full'] = docket_row['case_name_full']
    #docket_nodes[str(docket_row['id'])]['features']['slug'] = docket_row['slug']
    #docket_nodes[str(docket_row['id'])]['features']['docket_number'] = docket_row['docket_number']
    #docket_nodes[str(docket_row['id'])]['features']['docket_number_core'] = docket_row['docket_number_core']
    docket_nodes[str(docket_row['id'])]['features']['pacer_case_id'] = docket_row['pacer_case_id']
    #docket_nodes[str(docket_row['id'])]['features']['ia_date_first_change'] = docket_row['ia_date_first_change']
    #docket_nodes[str(docket_row['id'])]['features']['date_blocked'] = docket_row['date_blocked']
    ## add neighbors
    docket_nodes[str(docket_row['id'])]['neighbors']['opinion_cluster'] = docket2opinion_clusters[str(docket_row['id'])] if str(docket_row['id']) in docket2opinion_clusters else []
    docket_nodes[str(docket_row['id'])]['neighbors']['court'] = [str(docket_row['court_id'])]
    court2docket[str(docket_row['court_id'])].append(str(docket_row['id']))


# add court nodes
for idd, court_row in tqdm(court_raw_data.iterrows()):
    court_nodes[str(court_row['id'])] = {'features': {}, 'neighbors': {}}
    ## add features
    #court_nodes[str(court_row['id'])]['features']['date_modified'] = court_row['date_modified']
    #court_nodes[str(court_row['id'])]['features']['position'] = court_row['position']
    court_nodes[str(court_row['id'])]['features']['citation_string'] = court_row['citation_string']
    #court_nodes[str(court_row['id'])]['features']['short_name'] = court_row['short_name']
    court_nodes[str(court_row['id'])]['features']['full_name'] = court_row['full_name']
    #court_nodes[str(court_row['id'])]['features']['url'] = court_row['url']
    court_nodes[str(court_row['id'])]['features']['start_date'] = court_row['start_date']
    court_nodes[str(court_row['id'])]['features']['end_date'] = court_row['end_date']
    #court_nodes[str(court_row['id'])]['features']['jurisdiction'] = court_row['jurisdiction']
    #court_nodes[str(court_row['id'])]['features']['notes'] = court_row['notes']
    ## add neighbours
    court_nodes[str(court_row['id'])]['neighbors']['docket'] = court2docket[str(court_row['id'])] if str(court_row['id']) in court2docket else []

9592141it [14:42, 10874.15it/s]
29305192it [25:32, 19117.13it/s]


Miss citation cnt:580


8961877it [19:43, 7569.76it/s]
66164609it [2:12:19, 8333.13it/s]
3227it [00:00, 11205.05it/s]


In [9]:
## save graph
#pickle.dump({
#    'opinion_nodes': opinion_nodes,
#    'opinion_cluster_nodes': opinion_cluster_nodes,
#    'docket_nodes': docket_nodes,
#    'court_nodes': court_nodes
#}, open(os.path.join(save_dir, 'graph.pkl'),"wb"))

json.dump({
    'opinion_nodes': opinion_nodes,
    'opinion_cluster_nodes': opinion_cluster_nodes,
    'docket_nodes': docket_nodes,
    'court_nodes': court_nodes
}, open(os.path.join(save_dir, 'graph_raw.json'),"w"), indent = 4)

In [38]:
list(opinion_nodes.keys())[:5]

['7344185', '7344186', '7344187', '7344188', '7344189']

In [42]:
opinion_nodes['7344188']

{'features': {'date_created': '2022-07-26 00:22:09.289318+00',
  'date_modified': '2022-07-26 00:22:09.289335+00',
  'xml_harvard': '<opinion type="majority">\n<p id="AeGF">No opinion. Motion to dismiss appeal granted, with-$10 costs. Order filed. See, also, 102 N. Y. Supp. 929.</p>\n</opinion>',
  'plain_text': ''},
 'neighbors': {'opinion_cluster': ['7424460'],
  'reference': defaultdict(list, {}),
  'cited_by': defaultdict(list, {})}}